# Evaluate Pre-Trained Models on the Tripod Test Split

Used to generate figures and show how IoU degrades with time..

__Note:__ To maintain a high priority Colab user status such that sufficient GPU resources are available in the future, ensure to free the runtime when finished running this notebook. This can be done using 'Runtime > Manage Sessions' and click 'Terminate'.

In [ ]:
# Check if notebook is running in Colab or local workstation
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
    !pip install gputil
    !pip install psutil
    !pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()

try:
    # XXX: only one GPU on Colab and isn’t guaranteed
    gpu = GPUs[1]
    def printm():
        process = psutil.Process(os.getpid())
        print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
        print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
    printm() 

    # Check if GPU capacity is sufficient to proceed
    if gpu.memoryFree < 10000:
        print("\nInsufficient memory! Some cells may fail. Please try restarting the runtime using 'Runtime → Restart Runtime...' from the menu bar. If that doesn't work, terminate this session and try again later.")
    else:
        print('\nGPU memory is sufficient to proceeed.')
except:
    print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')

In [ ]:
if IN_COLAB:

    from google.colab import drive
    drive.mount('/content/drive')
    DATA_PATH = r'/content/drive/My Drive/Data'
    
    # cd into git repo so python can find utils
    %cd '/content/drive/My Drive/cciw-zebra-mussel/predict'

    sys.path.append('/content/drive/My Drive')
    
    # clone repo, install packages not installed by default
    !pip install pydensecrf

In [ ]:
import os
import os.path as osp

import glob

# for manually reading high resolution images
import cv2
import numpy as np

# for comparing predictions to lab analysis data frames
import pandas as pd

# for plotting
import matplotlib

'''
# enable LaTeX style fonts
matplotlib.rc('text', usetex=True)
import matplotlib.pyplot as plt
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
'''
import matplotlib.font_manager as fm
#!wget https://github.com/Phonbopit/sarabun-webfont/raw/master/fonts/thsarabunnew-webfont.ttf
fm.fontManager.ttflist += fm.createFontList(['thsarabunnew-webfont.ttf'])
matplotlib.rc('font', family='TH Sarabun New')
import matplotlib.pyplot as plt

# pytorch core library
import torch
# pytorch neural network functions
from torch import nn
# pytorch dataloader
from torch.utils.data import DataLoader

# for post-processing model predictions by conditional random field 
import pydensecrf.densecrf as dcrf
import pydensecrf.utils as utils

from tqdm import tqdm  # progress bar

# evaluation metrics
from sklearn.metrics import r2_score
from sklearn.metrics import jaccard_score as jsc

# local imports (files provided by this repo)
import transforms as T

# various helper functions, metrics that can be evaluated on the GPU
from task_3_utils import evaluate, evaluate_loss, eval_binary_iou, pretty_image, img_to_nchw_tensor

# Custom dataloader for rapidly loading images from a single LMDB file
from folder2lmdb import VOCSegmentationLMDB

In [ ]:
"""Confim that this cell prints "Found GPU, cuda". If not, select "GPU" as 
"Hardware Accelerator" under the "Runtime" tab of the main menu.
"""
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Found GPU,', device)

## 1. Load a pre-trained model checkpoint

The architecture is fully-convolutional network (FCN) 8s.

In [ ]:
sig = nn.Sigmoid()  # initializes a sigmoid function

In [ ]:
CID = '564349c'
SEED = 3

os.environ['DATA_PATH'] = '/scratch/gallowaa/'

if IN_COLAB:
    root = osp.join(
        DATA_PATH, 'Checkpoints/deeplabv3_resnet50_lr1e-01_wd5e-04_bs40_ep80_seed1')

else:
    root = osp.join(
        # os.environ['DATA_PATH'], 'cciw/logs/cmp-dataset/train_v120/deeplabv3_resnet50/lr1e-01/wd5e-04/bs40/ep80/seed1/checkpoint')
        os.environ['DATA_PATH'], 'cciw/logs/cmp-dataset/train_v120/deeplabv3_resnet50/lr1e-01/wd5e-04/bs40/ep80/seed%d/%s/checkpoint' % (SEED, CID))

ckpt_file = 'deeplabv3_resnet50_lr1e-01_wd5e-04_bs40_ep80_seed%d_epoch40.ckpt' % SEED

model_to_load = osp.join(root, ckpt_file)

print('Loading', model_to_load)

checkpoint = torch.load(model_to_load)

train_loss = checkpoint['trn_loss']
val_loss = checkpoint['val_loss']
print('==> Resuming from checkpoint..')
net = checkpoint['net']
last_epoch = checkpoint['epoch']
torch.set_rng_state(checkpoint['rng_state'])

# later appended to figure filenames
model_stem = ckpt_file.split('.')[0]

print('Loaded model %s trained to epoch ' % model_stem, last_epoch)
print(
    'Cross-entropy loss {:.4f} for train set, {:.4f} for validation set'.format(train_loss, val_loss))

sig = nn.Sigmoid()  # initializes a sigmoid function

net.eval()

## 7. i) Visualize Predictions on Whole Images

Here we manually load and preprocess the original images and png masks using OpenCV.

`root_path` -- will also be used in 

In [ ]:
#os.listdir('/scratch/ssd/gallowaa/cciw/dataset_raw/Test/Tripod/')

# to create the biofouling images under: 
# '/scratch/ssd/gallowaa/cciw/dataset_raw/Test/Tripod/1352/biofouling/'

'''
for f in jpeg_files[:32][::4]:
    #print(f)
    bgr_img = cv2.imread(f)
    image_stem = f.split('/')[-1].split('.')[0]
    img_name = image_stem + '_crop%d' % seek_y
    cv2.imwrite(osp.join(root_path, img_name + ".jpg"), bgr_img[seek_y:, :, :])
'''    

In [ ]:
if IN_COLAB:
    root_path = osp.join(DATA_PATH, 'ADIG_Labelled_Dataset/Test/Lab/')
else:
    root_path = '/scratch/ssd/gallowaa/cciw/dataset_raw/Test/Tripod/1352/'
    #root_path = '/scratch/ssd/gallowaa/cciw/dataset_raw/Test/Tripod/1352/biofouling'

jpeg_files = glob.glob(root_path + '*1.jpg')
png_files = glob.glob(root_path + 'biofouling/*.png')

jpeg_files.sort()
png_files.sort()

# Note: there is only one segmentation mask (PNG file) per Tripod site
print(len(jpeg_files)) 
print(len(png_files))

In [ ]:
#png_files

In [ ]:
#jpeg_files

In [ ]:
"""Set to True to save the model predictions in PNG format, 
otherwise proceed to predict biomass without saving images"""
SAVE_PREDICTIONS = False

if SAVE_PREDICTIONS:
    prediction_path = ''
    for t in root.split('/')[:-1]:
        prediction_path += t + '/'

    #prediction_path = osp.join(prediction_path, 'predictions-tripod')
    prediction_path = osp.join(prediction_path, 'predictions-tripod-biofouling')

    if not osp.exists(prediction_path):
        os.mkdir(prediction_path)
        
    print(prediction_path)

    # src is the training dataset, tgt is the testing dataset
    src = 'train_v120'
    tgt = 'Tripod'

In [ ]:
fontsize = 16

left = 0.02  # the left side of the subplots of the figure
right = 0.98   # the right side of the subplots of the figure
bottom = 0.05  # the bottom of the subplots of the figure
top = 0.95     # the top of the subplots of the figure
wspace = 0.15  # the amount of width reserved for space between subplots,
# expressed as a fraction of the average axis width
hspace = 0.1  # the amount of height reserved for space between subplots,
# expressed as a fraction of the average axis height

# LOM: Figure 5 (Tripod images)

In [ ]:
from task_3_utils import mask_and_preds_to_1hot

In [ ]:
# need to import colour_fmt_crop_and_resize
sys.path.append("..") # Adds higher directory to python modules path.

from utils.dataset_2_utils import colour_fmt_crop_and_resize

In [ ]:
colour_fmt_crop_and_resize

In [ ]:
sy = np.array([0, 1200])
scales = np.array([100, 125])

In [ ]:
# load mask
bgr_lab = cv2.imread(osp.join(root_path, png_files[0]))

# load a specific image at index i
i = 10

image_stem = jpeg_files[i].split('/')[-1].split('.')[0]
bgr_img = cv2.imread(osp.join(root_path, jpeg_files[i]))

#for seek_y in sy:
#for scale_percent in scales:
#seek_y = 1800
seek_x = 1200
seek_y = 0
scale_percent = 100

imgc, mask = colour_fmt_crop_and_resize(bgr_img, bgr_lab, seek_x, seek_y)

img_name = image_stem + '_crop%d' % seek_y

nchw_tensor = img_to_nchw_tensor(imgc, device)

with torch.no_grad():
    pred = sig(net(nchw_tensor)['out'])

pred_np = pred.detach().cpu().numpy().squeeze()

targets = torch.LongTensor(mask).to(device)
iou1 = eval_binary_iou(pred, targets).item()

'''
p_one_hot, t_one_hot = mask_and_preds_to_1hot(pred_np, mask)
iou2 = jsc(p_one_hot.reshape(1, -1),
           t_one_hot.reshape(1, -1), average='samples')
'''           
#print(seek_y, scale_percent, iou1, iou2)
print(seek_y, scale_percent, iou1)

p = (pred_np * 255).astype('uint8')
src2 = np.zeros(
    (p.shape[0], p.shape[1], 3), np.uint8)
src2[:, :, 2] = p
dst = cv2.addWeighted(imgc, 0.6, src2, 0.5, 0.5)
fig = plt.figure(figsize=(16, 12))
plt.imshow(dst)
plt.axis('off')
plt.tight_layout()
filename = src + '-' + tgt + '__' + image_stem + '__' + model_stem + \
    '_scale%d_iou1_%.3f_iou2_%.3f_crop%d' % (scale_percent, iou1, iou2, seek_y)
out_file = osp.join(prediction_path, filename)
print(out_file)

In [ ]:
cv2.imwrite(osp.join(root_path, img_name + ".jpg"), bgr_img[seek_y:, :, :])

# Biofouling figure

Series 1 - only use one mask

In [ ]:
bgr_lab = cv2.imread(osp.join(root_path, png_files[1]))

iou_list = []

seek_x = 1200
seek_y = 0 # for biofouling
#seek_y = 1200 # else for original 1352 images

for i in range(len(jpeg_files)):
    image_stem = jpeg_files[i].split('/')[-1].split('.')[0]
    bgr_img = cv2.imread(osp.join(root_path, jpeg_files[i]))
    
    imgc, mask = colour_fmt_crop_and_resize(bgr_img, bgr_lab, seek_x, seek_y)
    img = imgc.copy()

    nchw_tensor = img_to_nchw_tensor(imgc, device)

    with torch.no_grad():
        pred = sig(net(nchw_tensor)['out'])
    pred_np = pred.detach().cpu().numpy().squeeze()

    targets = torch.LongTensor(mask).to(device)
    iou1 = eval_binary_iou(pred, targets).item()
    print(iou1)
    iou_list.append(iou1)
    '''
    p_one_hot, t_one_hot = mask_and_preds_to_1hot(pred_np, mask)
    iou2 = jsc(p_one_hot.reshape(1, -1),
               t_one_hot.reshape(1, -1), average='samples')
    iou_list.append((iou1, iou2))
    '''

In [ ]:
#fname_npy = 'npy/Tripod_' + image_stem[4:] + '_biofouling_' + model_stem + "_" + CID + '.npy' # wrt may 25 fg cropped from img mask
fname_npy = 'npy/Tripod_' + image_stem[4:] + '_biofouling_fg_' + model_stem + "_" + CID + '.npy' # wrt may 25 fg
print(fname_npy)
np.save(fname_npy, np.asarray(iou_list))

Series 2 - for biofouling subset (10 images, 10 masks)

In [ ]:
iou_list = []

seek_y = 0 # for biofouling
seek_x = 1200

for i in range(len(jpeg_files)):
    image_stem = jpeg_files[i].split('/')[-1].split('.')[0]
    bgr_img = cv2.imread(osp.join(root_path, jpeg_files[i]))
    bgr_lab = cv2.imread(osp.join(root_path, png_files[i]))
    
    imgc, mask = colour_fmt_crop_and_resize(bgr_img, bgr_lab, seek_x, seek_y)
    img = imgc.copy()

    nchw_tensor = img_to_nchw_tensor(imgc, device)

    with torch.no_grad():
        pred = sig(net(nchw_tensor)['out'])
    pred_np = pred.detach().cpu().numpy().squeeze()

    targets = torch.LongTensor(mask).to(device)
    iou1 = eval_binary_iou(pred, targets).item()
    print(iou1)
    iou_list.append(iou1)

In [ ]:
fname_npy = 'npy/Tripod_biofouling_10_labels_propagated_' + model_stem + "_" + CID + '.npy'
print(fname_npy)
np.save(fname_npy, np.asarray(iou_list))

Series 3 - evaluate all images with most recent label from the 10 available

In [ ]:
from matplotlib.dates import datestr2num

In [ ]:
root_path = '/scratch/ssd/gallowaa/cciw/dataset_raw/Test/Tripod/1352/'

jpeg_files = glob.glob(root_path + '*1.jpg')
png_files = glob.glob(root_path + 'biofouling/*.png')

jpeg_files.sort()
png_files.sort()

# Note: lengths will be unequal here
print(len(jpeg_files)) 
print(len(png_files))

# assume len(png_files) < len(jpeg_files)
num_inserted = 0
for i in range(len(png_files)):
    png_date = png_files[i + num_inserted].split('/')[-1].split('_')[2]

    for j in range(i + num_inserted, len(jpeg_files)):
        jpeg_date = jpeg_files[j].split('/')[-1].split('_')[2]
        #print(j, png_date, jpeg_date)

        if datestr2num(png_date) > datestr2num(jpeg_date):
            png_files.insert(i + num_inserted, png_files[i + num_inserted - 1])
            num_inserted += 1
        else:
            # print('Exit!')
            break

    # print('\n')
    #print(png_files, len(png_files))
    # print('\n')

while len(png_files) < len(jpeg_files):
    png_files.append(png_files[-1])

assert len(jpeg_files) == len(png_files)

# Note: lengths should be equal here
print(len(jpeg_files)) 
print(len(png_files))

In [ ]:
bgr_img = cv2.imread(osp.join(root_path, jpeg_files[i]))
bgr_lab = cv2.imread(osp.join(root_path, png_files[i]))

In [ ]:
bgr_lab.shape

In [ ]:
plt.close('all')
fig, axes = plt.subplots(1, 1, figsize=(20, 15))
p = (pred_np * 255).astype('uint8')
src2 = np.zeros((p.shape[0], p.shape[1], 3), np.uint8)
src2[:, :, 2] = p
dst = cv2.addWeighted(img, 0.5, src2, 0.5, 0)
axes.imshow(dst)
#axes.imshow(img)
axes.axis('off')

plt.tight_layout()

if SAVE_PREDICTIONS:
    filename = src + '-' + tgt + '__' + image_stem + '__' + model_stem + '_iou_%.4f' % iou1
    out_file = osp.join(prediction_path, filename)
    #fig.savefig(out_file + '.jpg', format='jpeg')

# Split up image into smaller chunks

In [ ]:
#i = 1

iou_zoom_list = []

w_y = 1500
w_x = 2000
scale_percent = 150

bgr_lab = cv2.imread(osp.join(root_path, png_files[0]))

labc = cv2.cvtColor(bgr_lab, cv2.COLOR_BGR2RGB)

for i in range(len(jpeg_files)):

    image_stem = jpeg_files[i].split('/')[-1].split('.')[0]
    bgr_img = cv2.imread(osp.join(root_path, jpeg_files[i]))
    imgc = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
    composite_image = np.zeros((bgr_lab.shape[0], bgr_lab.shape[1]))

    for row in range(bgr_img.shape[0] // w_y):    
        for col in range(bgr_img.shape[1] // w_x):

            top = w_y * row
            bot = w_y * (row + 1)
            left  = w_x * col
            right = w_x * (col + 1)

            img = imgc[top:bot, left:right, :]
            lab = labc[top:bot, left:right]

            width = int(img.shape[0] * scale_percent / 100)
            height = int(img.shape[1] * scale_percent / 100)
            img = cv2.resize(img, (width, height)) # resize image
            lab = cv2.resize(lab, (width, height)) # resize image

            nchw_tensor = img_to_nchw_tensor(img, device)

            with torch.no_grad():
                pred = sig(net(nchw_tensor)['out'])
            pred_np = pred.detach().cpu().numpy()

            # OpenCV loads the PNG mask as indexed color RGB, 
            # we need to convert it to a binary mask. 
            # The `0' in labc[:, :, 0] is the R channel.
            mask = np.zeros((lab.shape[0], lab.shape[1]), dtype='float32')
            mask[lab[:, :, 0] == 128] = 1

            pred_np = pred_np.squeeze()        
            width = int(img.shape[0] * 100 / scale_percent)
            height = int(img.shape[1] * 100 / scale_percent)
            pred_np_native = cv2.resize(pred_np, (width, height)) # resize image
            composite_image[top:bot, left:right] = pred_np_native * 255

    pred = torch.FloatTensor((composite_image / 255)).to(device)
    mask = np.zeros((labc.shape[0], labc.shape[1]), dtype='float32')
    mask[labc[:, :, 0] == 128] = 1

    targets = torch.LongTensor(mask)
    targets = targets.to(device)

    outputs = pred.squeeze(1).round().long()
    intersection = (outputs & targets).float().sum((0, 1))
    union = (outputs | targets).float().sum((0, 1))
    iou = intersection / union
    print(iou.item())
    iou_zoom_list.append(iou)

In [ ]:
np.save('npy/Tripod_zoom' + str(scale_percent) + '_' + image_stem[4:] + '_biofouling_' + model_stem + '.npy', np.asarray(iou_zoom_list))

In [ ]:
#plt.figure(figsize=(20, 15))
#plt.imshow(composite_image)

In [ ]:
plt.close('all')
fig, axes = plt.subplots(1, 1, figsize=(20, 15))
#p = (pred_np * 255).astype('uint8')
p = composite_image.astype('uint8')
src2 = np.zeros((p.shape[0], p.shape[1], 3), np.uint8)
src2[:, :, 2] = p
dst = cv2.addWeighted(imgc, 0.5, src2, 0.5, 0)
axes.imshow(dst)
axes.axis('off')
plt.tight_layout()

if SAVE_PREDICTIONS:
    filename = src + '-' + tgt + '__composite' + image_stem + '__' + model_stem + '_iou_%.4f' % iou
    out_file = osp.join(prediction_path, filename)
    fig.savefig(out_file + '.jpg', format='jpeg')

In [ ]:
pred = torch.FloatTensor((composite_image / 255)).to(device)

In [ ]:
mask = np.zeros((labc.shape[0], labc.shape[1]), dtype='float32')
mask[labc[:, :, 0] == 128] = 1

targets = torch.LongTensor(mask)
targets = targets.to(device)

outputs = pred.squeeze(1).round().long()
intersection = (outputs & targets).float().sum((0, 1))
union = (outputs | targets).float().sum((0, 1))
iou = intersection / union
print(iou.item())

In [ ]:
w_y = 1500
w_x = 2000
scale_percent = 150

bgr_lab = cv2.imread(osp.join(root_path, png_files[0]))

labc = cv2.cvtColor(bgr_lab, cv2.COLOR_BGR2RGB)

i = 3
image_stem = jpeg_files[i].split('/')[-1].split('.')[0]
bgr_img = cv2.imread(osp.join(root_path, jpeg_files[i]))
imgc = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
composite_image = np.zeros((bgr_lab.shape[0], bgr_lab.shape[1]))

for row in range(bgr_img.shape[0] // w_y):    
    for col in range(bgr_img.shape[1] // w_x):

        top = w_y * row
        bot = w_y * (row + 1)
        left  = w_x * col
        right = w_x * (col + 1)

        img = imgc[top:bot, left:right, :]
        lab = labc[top:bot, left:right]

        width = int(img.shape[0] * scale_percent / 100)
        height = int(img.shape[1] * scale_percent / 100)
        img = cv2.resize(img, (width, height)) # resize image
        lab = cv2.resize(lab, (width, height)) # resize image

        nchw_tensor = img_to_nchw_tensor(img, device)

        with torch.no_grad():
            pred = sig(net(nchw_tensor)['out'])
        pred_np = pred.detach().cpu().numpy()

        # OpenCV loads the PNG mask as indexed color RGB, 
        # we need to convert it to a binary mask. 
        # The `0' in labc[:, :, 0] is the R channel.
        mask = np.zeros((lab.shape[0], lab.shape[1]), dtype='float32')
        mask[lab[:, :, 0] == 128] = 1

        pred_np = pred_np.squeeze()        
        width = int(img.shape[0] * 100 / scale_percent)
        height = int(img.shape[1] * 100 / scale_percent)
        pred_np_native = cv2.resize(pred_np, (width, height)) # resize image
        composite_image[top:bot, left:right] = pred_np_native * 255

pred = torch.FloatTensor((composite_image / 255)).to(device)
mask = np.zeros((labc.shape[0], labc.shape[1]), dtype='float32')
mask[labc[:, :, 0] == 128] = 1

targets = torch.LongTensor(mask)
targets = targets.to(device)

outputs = pred.squeeze(1).round().long()
intersection = (outputs & targets).float().sum((0, 1))
union = (outputs | targets).float().sum((0, 1))
iou = intersection / union
print(iou.item())

In [ ]:
#p = (pred_np * 255).astype('uint8')
p = composite_image.astype('uint8')

src2 = np.zeros((p.shape[0], p.shape[1], 3), np.uint8)
src2[:, :, 2] = p
dst = cv2.addWeighted(imgc, 0.6, src2, 0.5, 0.5)
fig = plt.figure(figsize=(16, 12))
plt.imshow(dst)
plt.axis('off')
plt.tight_layout()

# Calculate IoU between May 25 labels

For inter-annotator variability, and figure for supplement

In [ ]:
root_path = '/scratch/ssd/gallowaa/cciw/dataset_raw/Test/Tripod/1352/biofouling/'    
png_files = glob.glob(root_path + '*.png')
png_files.sort()
print(len(png_files))
lab_fg_1 = cv2.cvtColor(cv2.imread(osp.join(root_path, png_files[1])), cv2.COLOR_BGR2RGB)
mask_1 = np.zeros((lab_fg_1.shape[0], lab_fg_1.shape[1]), dtype='float32')
mask_1[lab_fg_1[:, :, 0] == 128] = 1

In [ ]:
plt.imshow(mask_1)

In [ ]:
root_path = '/scratch/ssd/gallowaa/cciw/dataset_raw/Test/Tripod/1352/'    
png_files = glob.glob(root_path + '*_final.png')
png_files.sort()
print(len(png_files))
lab_full = cv2.cvtColor(cv2.imread(osp.join(root_path, png_files[0])), cv2.COLOR_BGR2RGB)
seek_y = 1200
lab_fg_2 = lab_full[seek_y:]
mask_2 = np.zeros((lab_fg_2.shape[0], lab_fg_2.shape[1]), dtype='float32')
mask_2[lab_fg_2[:, :, 0] == 128] = 1

In [ ]:
m_1 = mask_1.astype('uint8')
m_2 = mask_2.astype('uint8')

t = (m_1.round() * 2).astype('uint8') + m_2

fig, ax = plt.subplots(1, 1, figsize=(40., 18.))
im = ax.imshow(t, cmap=plt.cm.get_cmap('magma', 4))
ax.axis('off')

labels = ['True Negative', 'False Negative', 'False Positive', 'True Positive']

# This function formatter will replace integers with target names
formatter = plt.FuncFormatter(lambda val, loc: labels[val])

# We must be sure to specify the ticks matching our target names
cbar = fig.colorbar(im, ax=ax, ticks=[0, 1, 2, 3], format=formatter, orientation="horizontal", shrink=0.5, pad=0.05, aspect=30, fraction=0.05)
cbar.ax.tick_params(labelsize=54)

plt.tight_layout()
plt.show()

In [ ]:
fig.savefig('may_25_interhuman.jpg')